# Χρήστος Ζαχαριουδάκης - 03400132 - ΕΔΕΜΜ

# Επιβλεπόμενη μάθηση: μελέτη dataset του αποθετηρίου OpenML

## **Ανάγνωση και επισκόπηση του dataset**

Στην αρχή της εργασίας θα φορτώσουμε το dataset **OML16**, από το αποθετήριο Μηχανικής Μάθησης **OpenML**, χρησιμοποιώντας το πακέτο **openml** της **Python**. Πριν γίνει αυτό, πρέπει να εγγραφούμε στην ιστοσελίδα [OpenML](https://www.openml.org/) και να αποκτήσουμε προσωπικό κλειδί API. Αφού γίνει αυτό, εκτελούμε τον παρακάτω κώδικα για να κατεβάσουμε το πακέτο **openml**:

In [1]:
!pip install openml

  Created wheel for openml: filename=openml-0.12.2-py3-none-any.whl size=137327 sha256=017fc8da6923f7404fc43242bf027c3e0ae318bc8de673e5ce428d1cd7806400
  Stored in directory: c:\users\xrigi\appdata\local\pip\cache\wheels\bc\16\10\f542aa38a32dfbdec03e4bb3c68b69dbbae98a013107333b39
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11731 sha256=8e998300ff4dfdb5b3db7f196af93e67795cc71e92ff44e89b731abfbd1ccab6
  Stored in directory: c:\users\xrigi\appdata\local\pip\cache\wheels\08\82\8b\5c514221984e88c059b94e36a71d4722e590acaae04deab22e
Successfully built openml liac-arff


Έπειτα χρησιμοποιούμε το πακέτο για να κατεβάσουμε το dataset, καθώς και τα μεταδεδομένα του, όπως τίτλο, περίληψη, πηγή και περιγραφή.

In [ ]:
import openml as oml
import warnings

# Suppress sklearn warnings
warnings.filterwarnings("ignore")

oml.config.apikey = 'fc544883e5a68a49c217ca42f908207e'
# Making sure we are using the right settings
oml.config.server = 'https://www.openml.org/api/v1/xml'

data_id = 16
dataset = oml.datasets.get_dataset(data_id)

Summary = "This is dataset '{}', the target feature is '{}'".format(dataset.name,dataset.default_target_attribute)
URL = dataset.url
description = dataset.description[:5000]
print(Summary,URL,description,sep ='\n')

dataset = dataset.get_data() 
data = dataset[0]
column_titles = dataset[3]
print("\nData has type of",type(data))
print("To dataset έχει διαστάσεις:", data.shape)
print("Attribute 1 has type of",data["att1"].dtype)
print("Class has type of",data["class"].dtype)

Σύμφωνα με τα παραπάνω, το dataset περιέχει **2000 δείγματα**, που περιγράφουν χειρόγραφα ψηφία με χρήση **64 χαρακτηριστικών** (64 Karhunen-Love coefficients) και το καθένα ανήκει σε μία κλάση από το 0 έως το 9 ανάλογα με ποιό είναι το ψηφίο που αναπαριστά. Επομένως έχουμε συνολικά **10 κλάσεις**. 

Σκοπός του dataset λοιπόν είναι η αναγνώριση χειρόγραφων ψηφίων με βάση αυτά τα 64 χαρακτηριστικά. Τα δεδομένα είναι αποθηκευεμένα ήδη σε μορφή pandas.DataFrame, οπότε δεν χρειάζεται κάποια μετατροπή. Όλα τα χαρακτηριστικά ειναι τύπου float (float64) και το class τύπου category. Tα δεδομένα του dataset είναι: 

In [ ]:
import pandas as pd
#pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option("display.max_rows", 10, "display.max_columns", 6)
print(data)


Όπως βλέπουμε, μέσα στο pandas.DataFrame, οι γραμμες είναι αριθμημένες και οι στήλες έχουν επικεφαλίδες της μορφής "att**i**", δηλαδή χαρακτηριστικό **i**. Τα χαρακτηριστικά είναι μη διατεταγμένα. Οι ετικέτες (labels) των κλάσεων βρίσκονται στην τελευταία στήλη με επιφαλίδα "class" και είναι με αύξουσα σείρα. Παρατηρούμε οτι η κάθε κλάση να αναπαρίσταται με τον αριθμό που αντιπροσωπεύει + 1. Δηλαδή το ψηφίο 0 ανήκει στην κλάση 1, το ψηφίο 1 στην 2 και ούτω κάθεξης. Αυτό μπορεί να προκαλέσει σύγχυση κατά την διάρκεια της εκπαίδευσης των μοντέλων που θα χρησιμοποιήσουμε, όποτε θα το αλλάξουμε:

In [ ]:
data['class'] = data['class'].astype('int64') # Convert column type from category to integer
ClassVal = {i+1:i for i in range(0,10)}
data["class"].replace(ClassVal, inplace=True)
print(data["class"])

Στην συνέχεια θα ελέγξουμε για απουσιάζοντες τιμές. Τόσο η εκτύπωση του dataset, όσο και οι πληροφορίες από την [πηγή του dataset](https://archive.ics.uci.edu/ml/datasets/Multiple+Features) μας ενημερώνουν ότι δεν υπάρχουν απουσιάζοντες τιμές. Ωστόσο θα κάνουμε επιπλέον ελέγχο:

In [ ]:
print(data.isnull().values.any())

Ο παραπάνω έλεγχος επιβεβαιώνει ότι δεν έχουμε απουσιάζουσες τιμές.

Σε αυτό το σημείο έχει ενδιαφέρον να δούμε το ποσοστό των δειγμάτων που ανήκουν σε κάθε κλάση:

In [ ]:
Percentages = data.groupby(['class']).size()/data.shape[0] * 100

for index, value in Percentages.iteritems():
    print("Class",index,": ",value,"%")

Βλέπουμε ότι έχουμε ένα τέλεια ισορροποιημένο dataset, με 10% των δειγμάτων σε κάθε κλάση, δηλαδή 200 δείγματα σε κάθε κλάση. Η παραπάνω πληροφορίες συνοψίζονται στον παρακάτω πίνακα markdown:

<div style="font-size: 50px">

| Ερώτηση| Απάντηση|
|:-:|:-:|
| Πρόβλημα dataset | Aναγνώριση χειρόγραφων ψηφίων με βάση 64 χαρακτηριστικά|
|   Μετατροπές στα αρχεία        | Καμία                |
| Πλήθος δειγμάτων | 2000 δείγματα|
|   Πλήθος χαρακτηριστικών       | 64 χαρακτηριστικά                |
|   Είδος χαρακτηριστικών       | Float              |
|   Διάταξη χαρακτηριστικών       | Μη διατεταγμένα            |
|  Επικεφαλίδες       | Υπάρχουν: {att1, ... , att64}               |
|  Αρίθμηση       | Υπάρχει μέσω του pandas.DataFrame             |
|  Ετικέτες       | Τελευταία κολόνα με τίτλο "class". Τιμές από 1 έως 10 (το αλλάξαμε σε 0 έως 9) με αύξουσα σειρά            |
|  Απουσιάζοντες τιμές       | Δεν υπάρχουν            |
| Πλήθος κλάσεων | 10 (μια για κάθε χειρόγραφο ψηφίο)|
| Ισορροπία dataset | Τέλεια ισορροποιημένο (10 % δειγμάτων σε κάθε κλάση)|

</div >

Στην συνέχεια, θα διασπάσουμε το dataset σε δείγματα εκπαίδευσης (70% του dataset) και δοκιμής (30% του dataset) με χρήση της συνάρτησης train_test_split της βιβιοθήκης scikit-learn:

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split

# Split our data
X_train, X_test, y_train, y_test = train_test_split(data.loc[:, 'att1':'att64'], data['class'], test_size=0.30,random_state=42)

#print(X_train)
#print(y_train)
#print("===========================")
#print(X_test)
#print(y_test)

Στην συνέχεια, θα εξετάσουμε την επίδοση ορισμένων ταξινομητων χώρις να εφαρμόσουμε βελτιστοποιήσεις (Επίδοση out-of-the-box), με τις παραμέτρους των ταξινομητών στις προεπιλεγμένες τους τιμές και εφαρμόζοντας ορισμένες βελτιστοποιήσεις.

## **Επίδοση out-of-the-box**

Οι ταξινομητές που θα μελετήσουμε είναι οι:

*   Dummy
*   Gaussian Naive Bayes
*   K-Nearest Neighbors
*   Logistic Regression
*   Multi-Layer Perceptron
*   Support Vector Machines

Θα χρησιμοποιήσουμε τις υλοποιήσεις τους, από την βιβλιοθήκη του scikit-learn. Επίσης για την αξιολόγηση της επίδοσης των ταξινομητών, θα χρησιμοποιήσουμε δύο μετρικές:

*   Accuracy = $\frac{Σωστές\;προβλέψεις}{Σύνολο\;δειγμάτων}$
*   F1-Score = $\frac{2 * (precision * recall)}{(precision + recall)}$ όπου precision = Positive Predictive Value (PPV) = $\frac{True\;Positive}{(True\;Positive + False\;Positive)}$ και
Recall = True Positive Rate (TPR) = $\frac{True\;Positive}{(True\;Positive + False\;Negative)}$

τις οποίες επισης λαμβάνουμε από το scikit-learn.

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

Classifiers = [DummyClassifier(),GaussianNB(),KNeighborsClassifier(),LogisticRegression(),MLPClassifier(),SVC()]
Accuracy = []
F1_Sc = []
Names = ["Dummy","GNB","KNN","LR","MLP","SVM"]

for i,clf in enumerate(Classifiers):
  clf.fit(X_train, y_train)  # Train the model  
  y_pred = clf.predict(X_test) # Make predictions on train samples (cheating)

  acc = accuracy_score(y_test, y_pred) * 100
  f1 = f1_score(y_test, y_pred, average='macro') * 100
  Accuracy.append(acc)
  F1_Sc.append(f1)
  
  # Print results
  print("{}. {}:  ".format(i+1,type(clf).__name__))
  print("\tAccuracy is {:.2f} %".format(acc))
  print("\tF1-Score is {:.2f} %".format(f1))

Παρατηρούμε εξαιρετικά αποτελέσματα (σχεδόν 100%) σε όλους τους ταξινομητές εκτός του Dummy, με την καλύτερη απόδοση να δίνεται από τον SVM classifier με πυρήνα "rbf". O SVM είναι ιδανικός για την ταξινόμηση δειγμάτων με πολλά χαρακτηριστικά και για αυτό το λόγο είναι αναμένόμενη η υψηλή επίδοσή του. Αντίθετα, ο Gaussian Naive Bayes υποθέτει (αφελώς) ότι τα χαρακτηριστικά των δειγμάτων είναι ανεξάρτητα και ότι τα δείγματα προέρχονται απο Gaussian κατανομή. Επόμενως λογικό είναι να έχει κάπως χαμηλότερη απόδοση.

Θα παρουσιάσουμε τα αποτελέσματα σε markdown table:

| Classifier | Accuracy | F1-Score |
| --- | --- | --- |
| Dummy | 8.17% | 1.51% |
| Gaussian Naive Bayes | 94.83% | 94.93% |
| K-Neighbors | 97.83% | 97.79% |
| Logistic Regression | 95.67% | 95.59% |
| Multi-Layer Perceptron | 97.33% | 97.41% |
| SVM | 98.17% | 98.13% |

και σε bar-plot με χρήση της βιβλιοθήκης **matplotlib**:

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(2, 1)
fig.suptitle('Metrics on train set')
fig.set_size_inches(18.5, 10.5)
ax1.bar(Names,Accuracy)
ax1.set_title("Accuracy")
ax2.bar(Names,F1_Sc)
ax2.set_title("F1-Score")

for ax in fig.get_axes():
    ax.label_outer()

plt.show()

## **Επίδοση με βελτιστοποιήσεις**

Θα εφαρμόσουμε ορισμένους μετασχηματισμούς στο dataset για χρήση στους ταξινομητές. Για την κατασκευή του μοντέλου θα βασιστούμε στην κλάση Pipeline του πακέτου imbalanced-learn:

In [ ]:
!pip install imbalanced-learn

Επίσης θα χρησιμοποιήσουμε διάφορες κλάσεις του πακέτου **scikit-learn** για την προεπεξεργασία των δεδομένων. Συγκεκριμένα θα χρησιμοποιήσουμε τις κλάσεις:

*   **VarianceThreshold** : Αφαιρεί τα χαρακτηριστικά στο training set που έχουν μηδενική διακύμανση (default) ή πολύ χαμηλή. Σε γενικές γραμμές αν η διακύμανση ενός χαρακτηριστικού εισόδου είναι πολύ χαμηλή, δεν μπορεί να προσφέρει σημαντικά στη διαχωριστική ικανότητα του ταξινομητή. Ειδικά στην περίπτωση που η διακύμανση είναι 0, δηλαδή το χαρακτηριστικό έχει σταθερή τιμή για όλα τα δείγματα εκπαίδευσης, δεν χρησιμεύει καθόλου στον ταξινομητή για να αποφασίσει αν ένα δείγμα ανήκει σε μία κλάση ή σε μια άλλη και επιπλέον μπορεί να δυσκολέψει άλλες διαδικασίες της προεπεξεργασίας όπως η κανονικοποίηση των χαρακτηριστικών.
*   **StandardScaler**: Θα εκτελέσει κανονικοποίηση των δειγμάτων (μέση τιμή μ=0 και διασπορά σ$^{2}$ =1) με βάση τα δείγματα εκπαίδευσης.
*   **PCA**: Εφαρμόζουμε εξαγωγή νέων χαρακτηριστικών σε ένα χώρο μικρότερων διαστάσεων (feature extraction) με την ανάλυση σε κύριες συνιστώσες (Principal Components Analysis). Με την μέθοδο αυτή, αναλύουμε τα δεδομένα σε κύριες συνιστώσες και δουλέυουμε με τελείως νέες, γραμμικά ασυσχέτιστες μεταβλητές μικρότερης διαστατικότητας. Η αναγκη μείωσης των χαρακτηριστικών έγκειται στο γεγονός ότι η απόδοση του μοντέλου αυξάνεται όταν τα δεδομένα εκπαίδευσης ειναι μεγαλύτερα σε μέγεθος και καλύτερα σε ποιότητα, αλλά μειώνεται όταν αυξάνονται οι διαστάσεις του dataset, δηλαδή ο αριθμός των χαρακτηριστικών. Το φαινόμενο αυτό είναι γνωστό ως κατάρα της διαστατικότητας (Curse of dimensionality).

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

selector = VarianceThreshold()
scaler = StandardScaler()
pca = PCA()
Classifiers = [DummyClassifier(),GaussianNB(),KNeighborsClassifier(),LogisticRegression(),MLPClassifier(),SVC()]

Σε αντίθεση με πριν, σε αυτό το στάδιο θα εξετάσουμε και τις τιμές των υπερπαραμέτρων των ταξινομητών ως προς ποιές παρέχουν την βέλτιστη απόδοση. Αυτό θα το επιτύχουμε με την τεχνική του Cross-Validation και συγκεκριμένα της κλάσης GridSearchCV του scikit-learn. Ουσιαστικά θα δοκιμάσουμε διάφορους συνδυασμούς, θα εκπαίδευσουμε το μοντέλο σε κάθε περίπτωση με Cross-Validation και θα κρατήσουμε τον ταξινομητή, που κρίνει η κλάση GridSearchCV, ότι είναι καλύτερος.

In [ ]:
import numpy as np

train_variance = np.array(X_train).var(axis=0)
print(train_variance)
print(np.max(train_variance))

Προσαρμόζουμε τις τιμές μας στο variance που παρατηρήσαμε:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import time

# Parameters
vthreshold = [0, 1, 5,10] # Variance Thresholds
n_components = [5,20, 50, 60] # PCA
scores = ['accuracy','f1_macro']

Στην συνέχεια θα περάσουμε το dataset μεσω του Pipeline για προεπεξεργασία και θα εκτελέσουμε 10-Fold Cross-Validation με κάθε ταξινομητή για να βρούμε τις βέλτιστες παραμέτρους για τους μετασχηματιστές και τον ταξινομητή:

**Dummy Classifier με παράμετρο την στρατηγική εκτίμησης προβλέψεων:**

In [ ]:
 # Classifier Parameters
strategy = ["stratified", "most_frequent", "prior", "uniform", "constant"] 

for score in scores:
  clf = DummyClassifier()  
  pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('DM', clf)], memory = 'tmp')
  estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, DM__strategy=strategy), cv=10, scoring=score)
  #print(estimator.get_params())    

  estimator.fit(X_train, y_train)
  print(estimator.best_estimator_)
  print(estimator.best_params_)

**Gaussian Naive Bayes Classifier με παράμετρο το ποσοστό της μεγαλύτερης διασποράς που προστίθεται στις διασπορές των δειγμάτων για σταθερότητα των υπολογισμών:**

In [ ]:
# Classifier Parameters
var_smoothing = [1e-9, 1e-8, 1e-6, 1e-4, 1e-2, 1e-1]  

for score in scores:
  clf = GaussianNB()  

  pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('GNB', clf)], memory = 'tmp')
  estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, GNB__var_smoothing=var_smoothing), cv=10, scoring=score)
  #print(estimator.get_params())    

  estimator.fit(X_train, y_train)
  print(estimator.best_estimator_)
  print(estimator.best_params_)

**K-Nearest Neighbors Classifier με παράμετρο τον αριθμό των γειτονικών δειγμάτων που εξετάζονται για να παρθεί η απόφαση για την ταξινόμηση:**

In [ ]:
# Classifier Parameters
k = [1, 3, 5, 11, 21, 31] 

for score in scores:  
  clf = KNeighborsClassifier()

  pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('kNN', clf)], memory = 'tmp')
  estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, kNN__n_neighbors=k), cv=10, scoring=score)

  estimator.fit(X_train, y_train)
  print(estimator.best_estimator_)
  print(estimator.best_params_)

**Logistic Regression Classifier με παράμετρο την μέθοδο regularization και το βάρος C των λανθασμένων ταξινομήσεων :**

In [ ]:
# Classifier Parameters
penalty = ['l1','l2' 'elasticnet', 'none']
C = [0,1,2,5,10]

for score in scores:
  clf = LogisticRegression()

  pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('LR', clf)], memory = 'tmp')
  estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, LR__penalty=penalty,LR__C = C), cv=10, scoring=score)

  estimator.fit(X_train, y_train)
  print(estimator.best_estimator_)
  print(estimator.best_params_)

**Multi-Layer Perceptron Classifier με παραμέτρους:**

*   Συνάρτηση ενεργοποίησης (activation function)
*   Βελτιστοποιητης (Optimizer)
*   Μέγεθος batch
*   Τιμή ρυθμού μάθησης

In [ ]:
# Classifier Parameters
activation = ['logistic','relu']
optimizer = ['adam']
batch_size = ['auto',200]
lr_val = [0.001,0.01]

for score in scores:
  clf = MLPClassifier()

  pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('MLP', clf)], memory = 'tmp')
  estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, MLP__activation = activation, MLP__solver = optimizer,MLP__batch_size = batch_size,MLP__learning_rate_init = lr_val), cv=10, scoring=score)

  estimator.fit(X_train, y_train)
  print(estimator.best_estimator_)
  print(estimator.best_params_)

O υπολογισμός του καλύτερου MLPClassifier καθυστερεί αρκετά, λόγω του πλήθους των υπερπαραμέτρων και της πολυπλοκότητας του μοντέλου.

**Support Vector Machines Classifier με παραμέτρους το βάρος C των λανθασμένων ταξινομήσεων, τον τύπο συνάρτηση πυρήνα και τον συντελεστή γ:**

In [ ]:
# Classifier Parameters
C = [0,1,2,5,10]
kernel =['linear', 'poly', 'rbf', 'sigmoid']
gamma = ['scale', 'auto'] 

for score in scores:
  clf = SVC()

  pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('SVM', clf)], memory = 'tmp')
  estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, SVM__C = C , SVM__kernel = kernel,SVM__gamma = gamma), cv=10, scoring=score)

  estimator.fit(X_train, y_train)
  print(estimator.best_estimator_)
  print(estimator.best_params_)

Με βάση τα παραπάνω αποτελέσματα οι καλύτερες τιμές παραμέτρων, με βάση την μετρική **accuracy**, για κάθε ταξινομητή είναι:

| Ταξινομητές | Παράμετροι |
| --- | --- |
| Dummy | {'DM__strategy': 'stratified', 'pca__n_components': 60, 'selector__threshold': 0} |
| Gaussian Naive Bayes | {'GNB__var_smoothing': 0.01, 'pca__n_components': 60, 'selector__threshold': 0} |
| K-Neighbors | {'kNN__n_neighbors': 3, 'pca__n_components': 20, 'selector__threshold': 5} |
| Logistic Regression | {'LR__C': 0, 'LR__penalty': 'none', 'pca__n_components': 60, 'selector__threshold': 0} | 
| Multi-Layer Perceptron | {'MLP__activation': 'relu', 'MLP__batch_size': 'auto', 'MLP__learning_rate_init': 0.001, 'MLP__solver': 'adam', 'pca__n_components': 20, 'selector__threshold': 5} | 
| SVM | {'SVM__C': 2, 'SVM__gamma': 'scale', 'SVM__kernel': 'rbf', 'pca__n_components': 20, 'selector__threshold': 5} |

Με βάση τα παραπάνω αποτελέσματα οι καλύτερες τιμές παραμέτρων, με βάση την μετρική **F1-macro**, για κάθε ταξινομητή είναι:

| Ταξινομητές | Παράμετροι |
| --- | --- |
| Dummy | {'DM__strategy': 'stratified', 'pca__n_components': 5, 'selector__threshold': 10} |
| Gaussian Naive Bayes | {'GNB__var_smoothing': 0.01, 'pca__n_components': 60, 'selector__threshold': 0} |
| K-Neighbors | {'kNN__n_neighbors': 3, 'pca__n_components': 20, 'selector__threshold': 5}|
| Logistic Regression | {'LR__C': 0, 'LR__penalty': 'none', 'pca__n_components': 60, 'selector__threshold': 0} | 
| Multi-Layer Perceptron | {'MLP__activation': 'relu', 'MLP__batch_size': 200, 'MLP__learning_rate_init': 0.001, 'MLP__solver': 'adam', 'pca__n_components': 50, 'selector__threshold': 1} | 
| SVM | {'SVM__C': 2, 'SVM__gamma': 'scale', 'SVM__kernel': 'rbf', 'pca__n_components': 20, 'selector__threshold': 5} |

Οι καλύτερες παράμετροι για τις δύο μετρικές είναι ίδιες με εξαίρεση τους Dummy και MLP classifiers. Στην συνέχεια λοιπον θα χρησιμοποιήσουμε τους καλύτερους ταξινομητές με βάση την μετρική **F1-Macro**.

**Dummy Classifier με παράμετρο την στρατηγική εκτίμησης προβλέψεων:**

In [ ]:
print("Dummy Classifier")

Accuracy_best = []
F1_sc_best = []

# Classifier Parameters
strategy = ["stratified"] 
vthreshold = [10] # Variance Thresholds
n_components = [5] # PCA

clf = DummyClassifier()  
pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('DM', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, DM__strategy=strategy), cv=10, scoring="f1_macro")  

start = time.time()
estimator.fit(X_train, y_train)
print("Train time is :",time.time()-start,"seconds")

start = time.time()
preds = estimator.predict(X_test)
print("Test time is :",time.time()-start,"seconds")

acc = accuracy_score(y_test, preds) * 100
f1 = f1_score(y_test, preds, average='macro') * 100
Accuracy_best.append(acc)
F1_sc_best.append(f1)
print("Accuracy is {}%".format(acc))
print("F1-Macro is {}%".format(f1))

**Gaussian Naive Bayes Classifier με παράμετρο το ποσοστό της μεγαλύτερης διασποράς που προστίθεται στις διασπορές των δειγμάτων για σταθερότητα των υπολογισμών:**

In [ ]:
print("Gaussian Naive Bayes Classifier")

# Classifier Parameters
var_smoothing = [1e-1]  
vthreshold = [0] # Variance Thresholds
n_components = [60] # PCA

clf = GaussianNB()  

pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('GNB', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, GNB__var_smoothing=var_smoothing), cv=10, scoring='f1_macro')
#print(estimator.get_params())    

start = time.time()
estimator.fit(X_train, y_train)
print("Train time is :",time.time()-start,"seconds")

start = time.time()
preds = estimator.predict(X_test)
print("Test time is :",time.time()-start,"seconds")
  
acc = accuracy_score(y_test, preds) * 100
f1 = f1_score(y_test, preds, average='macro') * 100
Accuracy_best.append(acc)
F1_sc_best.append(f1)
print("Accuracy is {}%".format(acc))
print("F1-Macro is {}%".format(f1))

**K-Nearest Neighbors Classifier με παράμετρο τον αριθμό των γειτονικών δειγμάτων που εξετάζονται για να παρθεί η απόφαση για την ταξινόμηση:**

In [ ]:
print("K-Nearest Neighbors Classifier")

# Classifier Parameters
k = [3]
vthreshold = [5] # Variance Thresholds
n_components = [20] # PCA

clf = KNeighborsClassifier()

pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('kNN', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, kNN__n_neighbors=k), cv=10, scoring="f1_macro")

start = time.time()
estimator.fit(X_train, y_train)
print("Train time is :",time.time()-start,"seconds")

start = time.time()
preds = estimator.predict(X_test)
print("Test time is :",time.time()-start,"seconds")
  
acc = accuracy_score(y_test, preds) * 100
f1 = f1_score(y_test, preds, average='macro') * 100
Accuracy_best.append(acc)
F1_sc_best.append(f1)
print("Accuracy is {}%".format(acc))
print("F1-Macro is {}%".format(f1))

**Logistic Regression Classifier με παράμετρο την μέθοδο regularization και το βάρος C των λανθασμένων ταξινομήσεων :**

In [ ]:
print("Logistic Regression Classifier")

# Classifier Parameters
penalty = ['none']
C = [0]
vthreshold = [0] # Variance Thresholds
n_components = [60] # PCA

clf = LogisticRegression()

pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('LR', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, LR__penalty=penalty,LR__C = C), cv=10, scoring='f1_macro')

start = time.time()
estimator.fit(X_train, y_train)
print("Train time is :",time.time()-start,"seconds")

start = time.time()
preds = estimator.predict(X_test)
print("Test time is :",time.time()-start,"seconds")
  
acc = accuracy_score(y_test, preds) * 100
f1 = f1_score(y_test, preds, average='macro') * 100
Accuracy_best.append(acc)
F1_sc_best.append(f1)
print("Accuracy is {}%".format(acc))
print("F1-Macro is {}%".format(f1))

**Multi-Layer Perceptron Classifier με παραμέτρους:**

*   Συνάρτηση ενεργοποίησης (activation function)
*   Βελτιστοποιητης (Optimizer)
*   Μέγεθος batch
*   Τιμή ρυθμού μάθησης

In [ ]:
print("MLP Classifier")

# Classifier Parameters
activation = ['relu']
optimizer = ['adam']
batch_size = [200]
lr_val = [0.001]
vthreshold = [1] # Variance Thresholds
n_components = [50] # PCA

clf = MLPClassifier()

pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('MLP', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, MLP__activation = activation, MLP__solver = optimizer,MLP__batch_size = batch_size,MLP__learning_rate_init = lr_val), cv=10, scoring='f1_macro')

start = time.time()
estimator.fit(X_train, y_train)
print("Train time is :",time.time()-start,"seconds")

start = time.time()
preds = estimator.predict(X_test)
print("Test time is :",time.time()-start,"seconds")

acc = accuracy_score(y_test, preds) * 100
f1 = f1_score(y_test, preds, average='macro') * 100
Accuracy_best.append(acc)
F1_sc_best.append(f1)
print("Accuracy is {}%".format(acc))
print("F1-Macro is {}%".format(f1))

**Support Vector Machines Classifier με παραμέτρους το βάρος C των λανθασμένων ταξινομήσεων, τον τύπο συνάρτηση πυρήνα και τον συντελεστη γ:**

In [ ]:
# Classifier Parameters
vthreshold = [5] # Variance Thresholds
n_components = [20] # PCA
C = [2]
kernel =['rbf']
gamma = ['scale'] 

clf = SVC()

pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('SVM', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, SVM__C = C , SVM__kernel = kernel,SVM__gamma = gamma), cv=10, scoring='f1_macro')

start = time.time()
estimator.fit(X_train, y_train)
print("Train time is :",time.time()-start,"seconds")

start = time.time()
preds = estimator.predict(X_test)
print("Test time is :",time.time()-start,"seconds")
  
acc = accuracy_score(y_test, preds) * 100
f1 = f1_score(y_test, preds, average='macro') * 100
Accuracy_best.append(acc)
F1_sc_best.append(f1)
print("Accuracy is {}%".format(acc))
print("F1-Macro is {}%".format(f1))

Σε αυτό το σημείο θα παρουσιάσουμε συνοπτικά και συγκριτικά την επίδοση των ταξινομητών για τις καλύτερες παραμέτρους τους σε σχέση με την out-of-the-box επίδοσή τους.

| Ταξινομητές | Accuracy (OOTB) | Accuracy (Best) |Μεταβολή accuracy| F1-Score (OOTB) | F1-Score (Best) | Μεταβολή F1-Score|Training time (Best) | Testing time (Best) | 
| --- | --- | --- | --- | --- |--- | --- |--- | --- |
| Dummy | 8.17% | 10.33% |+2.16% | 1.51% | 10.23% |+8.72% | 0.27s| 0.002s  |
| Gaussian Naive Bayes | 94.83%  | 92%|-2.83%| 94.93% |92.03%  |-2.90%|0.33s |0.003s  |
| K-Neighbors | 97.83% | 96.66% |-2.17%| 97.79% |96.60%  |-1.19%|0.32s |0.023s  |
| Logistic Regression | 95.67% | 91.66% |-4.01%| 95.59% |91.55%  |-4.04%|0.567s |0.002s  |
| Multi-Layer Perceptron |  97.33% |96.83%  |-1.33%| 97.41%  | 96.80% |-1.45%|10.42s | 0.003s |
| SVM |  98.17% | 97.5% |-1.67%| 98.13% |97.53%  |-0.6%|0.68s |0.039s  |

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(2, 1)
fig.suptitle('Metrics on train set')
fig.set_size_inches(18.5, 10.5)

x = np.arange(len(Names))  # the label locations
width = 0.25  # the width of the bars

difference = [list1_i-list2_i for list1_i, list2_i in zip(Accuracy, Accuracy_best)]

rects1 = ax1.bar(x,Accuracy,width,label = "OOTB")
rects2 = ax1.bar(x + width,Accuracy_best,width,label = "Best")
rects5 = ax1.bar(x + 2*width,difference,width,label = "Difference")
ax1.set_title("Accuracy")
ax1.set_xticks(x)
ax1.set_xticklabels(Names)
ax1.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)
autolabel(rects5)

difference = [list1_i-list2_i for list1_i, list2_i in zip(F1_Sc, F1_sc_best)]

rects3 = ax2.bar(x,F1_Sc,width,label = "OOTB")
rects4 = ax2.bar(x + width,F1_sc_best,width,label = "Best")
rects6 = ax2.bar(x + 2*width,difference,width,label = "Difference")
ax2.set_title("F1-Score")
ax2.set_xticks(x)
ax2.set_xticklabels(Names)
ax2.legend()

autolabel(rects3)
autolabel(rects4)
autolabel(rects6)

fig.tight_layout()
plt.show()

Αντίθετα από ότι αναμέναμε, παρατηρούμε μια ελαφρία μείωση στην απόδοση σε κάθε ταξινομητή. Μια ερμηνεία για αυτό το φαινόμενο είναι οτι η εφαρμογή μετασχηματισμών στα δεδομένα μας, αφαίρεσε πληροφορία χρήσιμη για την ταξινόμηση των δειγμάτων. Ο καλύτερος ταξινομήτης και στις δύο περιπτώσεις, με βάση αμφότερες μετρικές, είναι ο **SVM (με kernel = "rbf")** και ο χειρότερος, με εξαίρεση τον Dummy, είναι ο **Gaussian Naive Bayes**.

Σε ότι αφορά τους χρόνους, ο χρόνος εκπαίδευσης είναι πάντα μεγαλύτερος του χρόνου δοκιμής. Αυτό είναι λογικό, καθως στην εκπαίδευση, το μοντέλο πρέπει να μάθει την κατανομή των δειγμάτων, ενώ στην δοκιμή εκτελεί προβλέψεις με βάση αυτά που έχει μάθει. Ο μεγαλύτερος χρόνος εκπαίδευσης ανήκει στο MLP classifier. Επίσης είναι αναμενόμενο, καθώς είναι ο πιο σύνθετος από τους ταξινομητές που χρησιμοποιήσαμε.

Στη συνέχεια, θα σχεδιάσουμε τους πίνακες σύγχυσης για αυτούς τους δύο ταξινομητές, με χρήση του πακέτου **seaborn**.

In [ ]:
!pip install seaborn

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

print("################### SVM #############################")
vthreshold = [5] # Variance Thresholds
n_components = [20] # PCA
C = [2]
kernel =['rbf']
gamma = ['scale'] 

clf = SVC()

pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('SVM', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, SVM__C = C , SVM__kernel = kernel,SVM__gamma = gamma), cv=10, scoring=score)

estimator.fit(X_train, y_train)
best = estimator.predict(X_test)

conf_best = confusion_matrix(y_test, best)
print(conf_best,end='\n')

print("################### GNB #############################")

# Classifier Parameters
var_smoothing = [1e-1]  
vthreshold = [0] # Variance Thresholds
n_components = [60] # PCA

clf = GaussianNB()  

pipe = Pipeline(steps=[('selector', selector), ('scaler', scaler), ('pca', pca), ('GNB', clf)], memory = 'tmp')
estimator = GridSearchCV(pipe, param_grid = dict(selector__threshold=vthreshold, pca__n_components=n_components, GNB__var_smoothing=var_smoothing), cv=10, scoring='f1_macro')
#print(estimator.get_params())    

estimator.fit(X_train, y_train)
worst = estimator.predict(X_test)

conf_worst = confusion_matrix(y_test, worst)
print(conf_worst,end='\n')

fig, axs = plt.subplots(1,2,constrained_layout=True)
fig.suptitle('Confusion matrices')
sns.heatmap(conf_worst,ax = axs[0])
axs[0].set_title('GNB confusion matrix')
sns.heatmap(conf_best,ax = axs[1])
axs[1].set_title('SVM confusion matrix')

Επειδή και οι δύο αλγόριθμοι αποδίδουν πολύ καλά (άνω του 90%), οι πίνακες σύγχυσής τους είναι όμοιοι και διαγώνιοι, ειδικά ο SVM.

Από την παραπάνω μελέτη, ο καλύτερος ταξινομητής και αυτός που θα προτίμαμε τελικά για το συγκεκριμένο dataset, είναι ο SVM με kernel 'rbf', ο οποίος μας δίνει μέγιστο accuracy και F1-Score, 98.17% και 98.13% αντίστοιχα, ποσοστά εξαιρετικά κοντά στο 100%. Ένας λόγος που αποδίδει καλά, είναι επειδή εξ ορισμού, αποδίδει καλά σε datasets με μεγάλο αριθμό χαρακτηριστικών, όπως αυτό που χρησιμοποιήσαμε στην παρούσα άσκηση.